# # Plotting evaluation results.

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import platform
import matplotlib as mpl
import random
from copy import copy
import re

import matplotlib.animation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp

from pathlib import Path

from relnet.agent.baseline.baseline_agent import *
from relnet.agent.rnet_dqn.rnet_dqn_agent import RNetDQNAgent

from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.experiment_conditions import *
from relnet.evaluation.file_paths import FilePaths
from relnet.visualization import *

from relnet.state.network_generators import create_generator_instance, get_graph_ids_to_iterate
from relnet.data_wrangling.geocoder import Geocoder

storage = EvaluationStorage()

considered_agents = {RandomAgent.algorithm_name,
                       GreedyAgent.algorithm_name,
                       LowestDegreeProductAgent.algorithm_name,
                       FiedlerVectorAgent.algorithm_name,
                       EffectiveResistanceAgent.algorithm_name,
                       RNetDQNAgent.algorithm_name
                       }

considered_agents_models = [RNetDQNAgent.algorithm_name]
considered_agents_nondet = [RandomAgent.algorithm_name, RNetDQNAgent.algorithm_name]

cols_order = ['random', 'lowest_degree_product', 'fiedler_vector', 'effective_resistance', 'greedy', 'rnet_dqn', 'rnet_dqn_best']

fp = FilePaths('/experiment_data', 'aggregate')
exp_id = 'real_world'


In [ ]:
results = storage.get_evaluation_data('model', exp_id)
results_df = pd.DataFrame(results)
results_df.drop(columns=['network_size'], inplace=True)
results_df['algorithm'] = pd.Categorical(results_df['algorithm'], cols_order)
results_df = results_df.sort_values("algorithm")

# Generating table of results -- several experiments, varying sizes.

In [ ]:
def compute_ci(data, confidence=0.95):
    a = np.array(data)
    n = len(a)
    se = sp.stats.sem(a)
    h = se * sp.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

def highlight_max(s):
    #s = s['mean']
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

csv_results_file = fp.figures_dir / f"results_evaluation_models_rw_individual.csv"
pivot = pd.pivot_table(results_df, values='cummulative_reward', 
                       columns=['algorithm'], 
                       index=['objective_function', 'network_generator', 'graph_id'],
                       aggfunc=np.mean)
pivot

rnet_dqn_df = results_df[results_df['algorithm'] == 'rnet_dqn']
maxcol = pd.pivot_table(rnet_dqn_df, values='cummulative_reward', 
                       columns=['agent_seed'], 
                       index=['objective_function', 'network_generator', 'graph_id'],
                       aggfunc=np.mean)
maxes = maxcol.max(axis=1)
pivot["rnet_dqn_best"] = maxes

nondet_df = results_df[results_df['algorithm'].isin(considered_agents_nondet)]
nondet_means_df = pd.pivot_table(nondet_df, values='cummulative_reward', 
                       columns=['algorithm', 'agent_seed'], 
                       index=['objective_function', 'network_generator', 'graph_id'],
                       aggfunc=np.mean)




format_ci_dict = {}
for agent_name in considered_agents_nondet:
    cis = nondet_means_df[agent_name].apply(compute_ci, axis=1)
    pivot.columns = pivot.columns.add_categories([agent_name + "_ci"])
    pivot[agent_name + "_ci"] = cis
    format_ci_dict[agent_name + "_ci"] = (lambda x: "±{:.3f}".format(abs(x)))


means_across_runs = nondet_means_df.mean(axis=1)
subset = pivot[['rnet_dqn', 'rnet_dqn_ci']]
subset.style.format("{:.4f}").format(format_ci_dict)
pivot.to_csv(csv_results_file)
pivot.style.format("{:.3f}").format(format_ci_dict)

In [ ]:
latex_df = pivot.copy()
for nondet_agent in considered_agents_nondet:
    colname_ci = f"{nondet_agent}_ci"
    latex_df[nondet_agent] = latex_df.agg(lambda x: f"{x[nondet_agent]:.3f}±{x[colname_ci]:.3f}", axis=1)
    latex_df.drop(columns=[colname_ci], inplace=True)
    
latex_df = latex_df[cols_order]
row_maxes = latex_df.max(axis=1)

repl_cols = copy(agent_display_names)
latex_df.rename(columns=repl_cols, inplace=True)
texfile =  str(fp.figures_dir / f"results_evaluation_models_rw_individual.tex")
fh = open(texfile, 'w')
table_colformat = "ccl|" + ''.join(['c'] * len(cols_order))
latex_df.to_latex(buf=fh, float_format="{:0.3f}".format, column_format=table_colformat)
fh.close()



replace_dict = {
    r"objective\\_function": r"Objective",
    r"network\\_generator" : r"Dataset",
    r"algorithm" : r"",
    r"RNet–DQN\s+&" : "\\multicolumn{2}{c}{RNet–DQN}",
    r"rnet\\_dqn\\_best" : r"",
    r"random\\_removal": r"$\\mathcal{F}_{random}$",
    r"targeted\\_removal": r"$\\mathcal{F}_{targeted}$",
    r"graph\\_id": r"Country",
    r"euroroad": r"Euroroad",
    r"scigrid": r"Scigrid",
    r"±(\d+\.\d+)": r"\\tiny{$\\pm\g<1>$}"
}


for cc, name in Geocoder.COUNTRY_CODE_TO_NAME.items():
    cc_padded = rf"&\s+{cc}\s+&"
    name_padded = rf"& {name} &"
    replace_dict[cc_padded] = name_padded
    
print(replace_dict)

with open(texfile, 'r') as f:
    raw_content = f.read()

processed_content = raw_content
for orig, targ in replace_dict.items():
    processed_content = re.sub(orig, targ, processed_content, flags = re.M)
    
with open(texfile, 'w') as g:
    g.write(processed_content)
    
with open(texfile, 'r') as f:
    content_lines = f.readlines()
    
content_out = []    
for i, line in enumerate(content_lines):
    if i == 1 or i == (len(content_lines) - 2): 
        continue
    if i == 3:
        splits = line.split("&")
        splits[-2] = " \\textit{avg}"
        splits[-1] = " \\textit{best} \\\\\n"
        content_out.append("&".join(splits))
        continue
    
    content_out.append(line)

with open(texfile, 'w') as g:
    g.write("".join(content_out))
    